Importamos todas las librerias necesarias en el notebook

In [1]:
import matplotlib.pyplot as plt
import numpy
import pandas
import seaborn
import sys
from scipy import stats

In [2]:
seaborn.__version__

'0.9.0'

Leemos el dataset correspondiente al primer practico.

In [3]:
dataset = pandas.read_csv('meli_dataset_20190426.csv')
row0 = dataset.shape[0]
row0

499948

In [4]:
dataset.head()

,ITEM_ID,SHP_WEIGHT,SHP_LENGTH,SHP_WIDTH,SHP_HEIGHT,ATTRIBUTES,CATALOG_PRODUCT_ID,CONDITION,DOMAIN_ID,PRICE,SELLER_ID,STATUS,TITLE
0,M1CQ76ZT5W,NaN,NaN,NaN,NaN,NaN,H53U1H7Q5G,NaN,NaN,NaN,NaN,404,NaN
1,SN7ISIGQ9J,235.0,25.0,25.0,10.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-SKIN_CARE_SUPPLIES,68.0,QF4OJMYQ9Q,active,Ácido Hidroquinona 20% 30g + Sabonete Pré Pe...
2,JGEV50GW2U,1757.0,23.0,17.0,16.0,"[{'id': 'ACCESSORIES_INCLUDED', 'name': 'Acess...",YRBDJR6T7Y,new,MLB-NEBULIZERS,145.9,WEE71CZC2Q,active,Inalador E Nebulizador Infantil Nebdog Superfl...
3,JGEV50GW2U,1748.0,23.0,17.0,16.0,"[{'id': 'ACCESSORIES_INCLUDED', 'name': 'Acess...",YRBDJR6T7Y,new,MLB-NEBULIZERS,145.9,WEE71CZC2Q,active,Inalador E Nebulizador Infantil Nebdog Superfl...
4,JGEV50GW2U,NaN,NaN,NaN,NaN,"[{'id': 'ACCESSORIES_INCLUDED', 'name': 'Acess...",YRBDJR6T7Y,new,MLB-NEBULIZERS,145.9,WEE71CZC2Q,active,Inalador E Nebulizador Infantil Nebdog Superfl...


**1 - Eliminar valores cuyo status sea `404` , luego eliminar la columna `status` del dataset ya que solo es útil para limpieza.**

In [5]:
indices = dataset[ dataset['STATUS'] == '404' ].index
dataset.drop(indices , inplace=True)

In [6]:
row1 = dataset.shape[0]
row0-row1

78361

In [7]:
row0/(row0-row1)

6.380061510190018

Vemos que 78361 filas fueron removidas. Esto es un 6 porciento de data set original de casi 500 mil filas.

2 - Eliminar los valores NaN de las columnas con prefijo `SHP_`. Estas son aquellas que representan o peso o dimensiones de un item.

In [8]:
indices = dataset[dataset['SHP_WEIGHT'].isna() | dataset['SHP_LENGTH'].isna() | 
                  dataset['SHP_WIDTH'].isna() | dataset['SHP_HEIGHT'].isna() ].index
dataset.drop(indices , inplace=True)
row2 = dataset.shape[0]

In [9]:
print('Nos quedan ',row2,' filas. Fueron removidas ',row1-row2, '. Esto es un ',row2/(row1-row2),'porciento.')

Nos quedan  296325  filas. Fueron removidas  125262 . Esto es un  2.365641615174594 porciento.


**3-Agrupar por item id y calcular mediana de peso y medidas. De esta forma debería quedar una única fila por cada item_id.**


In [10]:
listColumns = list(dataset.columns)
columns_dic={}
for item in listColumns:
    if item.startswith('SHP_'):
        columns_dic[item] = 'median'
    else:
        columns_dic[item] = 'first'
dataset = dataset.groupby('ITEM_ID').agg(columns_dic)


In [11]:
dataset.describe()

,SHP_WEIGHT,SHP_LENGTH,SHP_WIDTH,SHP_HEIGHT,PRICE
count,236443.00000,236443.000000,236443.000000,236443.000000,2.106590e+05
mean,1818.96622,31.398151,21.251572,11.503536,1.958341e+03
std,3199.67595,18.469790,11.157975,8.234870,6.638232e+05
min,1.00000,0.000000,0.000000,0.000000,1.000000e-01
25%,250.00000,20.000000,13.000000,5.000000,4.790000e+01
50%,650.00000,25.000000,20.000000,10.000000,9.999000e+01
75%,1883.75000,36.000000,25.000000,16.000000,1.799000e+02
max,50000.00000,288.200000,115.000000,105.000000,3.032487e+08


**4-Parsear la columna de atributos y extraer a columnas propias aquellos atributos cuyo `id` sea `BRAND` o `MODEL`. Estos atributos representan marca o modelo que el vendedor del item ingresó en la publicación. [Opcional] No es necesario limitarse a estos dos atributos, se puede probar quedarse con los N atributos más frecuentes.**



In [12]:
dataset.head()

,ITEM_ID,SHP_WEIGHT,SHP_LENGTH,SHP_WIDTH,SHP_HEIGHT,ATTRIBUTES,CATALOG_PRODUCT_ID,CONDITION,DOMAIN_ID,PRICE,SELLER_ID,STATUS,TITLE
ITEM_ID,,,,,,,,,,,,,
A002DG7EAZ,A002DG7EAZ,812.0,36.0,32.0,12.0,NaN,H53U1H7Q5G,NaN,NaN,NaN,U2T0EY02XB,under_review,Apresentador Multimídia Wireless Logitech R400...
A00SG33UIH,A00SG33UIH,2320.0,16.0,11.0,4.0,"[{'id': 'AUTHOR', 'name': 'Autor', 'value_id':...",H53U1H7Q5G,new,MLB-BOOKS,149.99,BCZWFNME44,active,Apostila Trt-sp 2018 - Analista Jud. Área Apo...
A00VIC9XL7,A00VIC9XL7,213.0,16.0,13.0,10.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-BABY_GROOMING_KITS,329.00,T2JY69NPBA,active,Wetstop 3 Alarme Miccional Xixi Na Cama Enurese
A00VM7MP9F,A00VM7MP9F,175.0,25.0,20.0,15.0,"[{'id': 'ITEM_CONDITION', 'name': 'Condição do...",H53U1H7Q5G,new,MLB-BICYCLE_BOTTLE_CAGES,45.00,YA6XOOJU39,active,Suporte De Garrafa Zefal Wiiz Para Bicicleta
A00W1VSE3K,A00W1VSE3K,82.0,30.0,15.0,5.0,"[{'id': 'ALARM', 'name': 'Com alarme', 'value_...",H53U1H7Q5G,new,MLB-PEDOMETERS,31.98,DCLDPQAY43,active,Relógio Marcador De Passos Distancia E Caloria...


**1-Definir el target. En nuestro caso, agregar una columna no_maquinable la cual tendrá valor de verdad True si alguna de las dimensiones es > 70 y False en caso contrario**

In [13]:
labels=(dataset['SHP_LENGTH']>70) | (dataset['SHP_WIDTH'] >70) | (dataset['SHP_HEIGHT'] >70)

**2-Hacer un split de train/test del dataset limpio. Se recomienda usar train_test_split con 80% para training y 20% para test.**

In [14]:
!pip install sklearn

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
data_train, data_test, label_train, label_test = train_test_split(dataset, labels, test_size = 0.2)

**3-Elegir entre 2 modelos distintos vistos en clases (Sugerencia: Perceptron y Naive Bayes) y entrenar los modelos.**


In [17]:
from sklearn.feature_extraction.text import CountVectorizer

In [18]:
from nltk.corpus import stopwords

In [19]:
import string 

In [20]:
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('portuguese')]

In [21]:
vectorizer = CountVectorizer(analyzer=text_process)

In [22]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/pbrizuela/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
vectorizer.fit(dataset['TITLE'])

CountVectorizer(analyzer=<function text_process at 0x7fc8e3e81598>,
        binary=False, decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=1.0,
        max_features=None, min_df=1, ngram_range=(1, 1), preprocessor=None,
        stop_words=None, strip_accents=None,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, vocabulary=None)

In [ ]:
print(vectorizer.vocabulary_)

In [42]:
# encode document
vector = vectorizer.transform(dataset['TITLE'])

In [43]:
print(dataset['TITLE'].shape)
vector

(236443,)


<236443x115816 sparse matrix of type '<class 'numpy.int64'>'
	with 1866955 stored elements in Compressed Sparse Row format>

In [44]:
data_train, data_test, label_train, label_test = train_test_split(vector, labels, test_size = 0.2)

Perceptron

In [45]:
from sklearn.linear_model import Perceptron

In [46]:
clf = Perceptron(tol=1e-3, random_state=0)

In [49]:
clf.fit(data_train, label_train.astype(int))

Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
      fit_intercept=True, max_iter=None, n_iter=None, n_iter_no_change=5,
      n_jobs=None, penalty=None, random_state=0, shuffle=True, tol=0.001,
      validation_fraction=0.1, verbose=0, warm_start=False)

In [51]:
clf.score(data_test, label_test.astype(int))

0.9406415868383768

Naive Bayes

In [ ]:
labels.astype(int)

In [26]:
from sklearn.naive_bayes import MultinomialNB

In [27]:
clf = MultinomialNB()

In [28]:
clf.fit(data_train, label_train.astype(int))

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [29]:
predicted = clf.predict(data_test)

In [38]:
label_test.shape

(47289,)

In [37]:
label_test.sum()

2063

In [33]:
predicted.sum()

1659

In [30]:
 numpy.mean(predicted == label_test.astype(int))

0.948148618071856

In [31]:
label_test.describe()

count     47289
unique        2
top       False
freq      45226
dtype: object

**4-Evaluar y reportar métricas (Sugerimos: Confusion matrix, precision, recall y f1-score). Puede ser de utilidad ver: link1, link2.**

**5-Splitear el training set en train y validación (Sugerencia 80-20). Re-entrenar usando el conjunto de train y evaluar en el conjunto de validación con distintos hiper-parámetros los distintos modelos elegidos. Reportar las métricas de las distintas pruebas y describir cual es la mejor elección de hiper-parámetros. Explicar las métricas entendiendo el desbalance de clases del target. Reportar las métricas del mejor modelo sobre el conjunto de test y compararlo con los modelos entrenados en el punto 3.**

In [ ]:
dataset['TARGET'].sample(100)